In [1]:
import pandas as pd
import numpy as np
import os


import plotly.express as px
import plotly.figure_factory as ff
pd.options.plotting.backend = 'plotly'

ModuleNotFoundError: No module named 'plotly'

# Lecture 08 – Data on the internet

### Agenda

- Introduction to HTTP.
- Making HTTP requests.
- Data formats.
- APIs and scraping
- The anatomy of HTML documents.
- Parsing HTML using Beautiful Soup

## Introduction to HTTP



### Data sources

* Sometimes, the data you need doesn't exist in "clean" `.csv` files.

* **Solution:** Collect your own data!
    - Design and administer your own survey or run an experiment.
    - Find related data on the internet.

- The internet contains **massive** amounts of historical record; for most questions you can think of, the answer exists somewhere on the internet.

### Collecting data from the internet

- There are two ways to programmatically access data on the internet:
    - through an API.
    - by scraping.

### HTTP

- HTTP stands for **Hypertext Transfer Protocol**.
    - It was developed in 1989 by Tim Berners-Lee (and friends).

- It is a **request-response** protocol.
    - Protocol = set of rules.

- HTTP allows...
    - computers to talk to each other over a network.
    - devices to fetch data from "web servers".

- The "S" in HTTPS stands for "secure".

### ARPANET 

Advanced Research Projects Agency Network (Many of the protocols used by computer networks today were developed for ARPANET, and it is considered the forerunner of the modern internet.)

<center><img src='imgs/arpanet.png'></center>

### The request-response model

HTTP follows the **request-response** model.

<center><img src='imgs/req-response.png' width=600></center>

- A **request** is made by the **client**.

- A **response** is returned by the **server**.

- **Example:** Tiktok 🎥.
    - Your phone's mobile App, a **client**, makes an HTTP **request** to view a video.
    - The **server**, Tiktok, is a computer that is sitting somewhere else.
    - The server returns a **response** that contains the video.

### Request methods

The request methods you will use most often are `GET` and `POST`; see [Mozilla's web docs](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods) for a detailed list of request methods.    

- `GET` is used to request data **from** a specified resource.

- `POST` is used to **send** data to the server. 
    - e.g. uploading a video to Tiktok or entering credit card information on Amazon.

### Example `GET` request

Below is an example `GET` HTTP request made by a browser when accessing [bing.com](https://bing.com).

```HTTP
authority: assets.msn.cn
:method: GET
:path: /bundles/v1/bingHomepage/latest/card-actions-wc.544845754c7853b4c833.js
:scheme: https
accept: */*
accept-encoding: gzip, deflate, br
accept-language: zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7
if-none-match: 0x8DB1694BFE94BE8
origin: https://www.bing.com
referer: https://www.bing.com/
sec-ch-ua: "Chromium";v="110", "Not A(Brand";v="24", "Google Chrome";v="110"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "macOS"
sec-fetch-dest: script
sec-fetch-mode: cors
sec-fetch-site: cross-site
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0
```

-  "request line"
-  "header fields". Header fields contain metadata. 

- We _could_ also provide a "body" after the header fields.

- To see HTTP requests in Google Chrome, follow [these steps](https://mkyong.com/computer-tips/how-to-view-http-headers-in-google-chrome/).

### Example `GET` response



### Consequences of the request-response model

- When a request is sent to view content on a webpage, the server must:
    - process your request (i.e. prepare data for the response).
    - send content back to the client in its response.

- Remember, servers are computers. 
    - Someone has to pay to keep these computers running.
    - **This means that every time you access a website, someone has to pay.**

## Making HTTP requests

### Making HTTP requests

We'll see two ways to make HTTP requests outside of a browser:

- From the command line, with `curl`.

- **From Python, with the `requests` package.**

### Making HTTP requests using `curl`

[`curl`](https://curl.haxx.se/docs/httpscripting.html) is a **command-line tool** that sends HTTP requests, like a browser.

1. The client, `curl`, sends a HTTP request. 
2. The request contains a method (e.g. `GET` or `POST`).
3. The HTTP server responds with:
    - a status line, indicating if things went well, 
    - response headers, and
    - (usually) a response body, containing the requested data.

### Example: `GET` requests via `curl`

- By default, `curl` issues a `GET` request.

```zsh
# `-v` is short for verbose
curl -v https://httpbin.org/html 
```

- Remember, you can run command-line commands in a Jupyter Notebook by placing a `!` before them. Let's try that here.

In [ ]:
# Compare the output to what you see when you go to https://httpbin.org/html in your browser!
!curl -v https://httpbin.org/html

### Queries in a `GET` request

- In order to request more specific information, we can include a **query string** in the URL. `?` begins a query.

<a href="https://www.bing.com/search?q=shanghai"><pre>
https://www.bing.com/search?q=shanghai
</pre></a>

- This method works well when sending small amounts of data; we will use a similiar technique when working with APIs (coming later!).

- Be on the lookout for query strings in URLs you share on social media!

In [ ]:
!curl -v http://cn.bing.com/search?q=shanghai

### Making HTTP requests using `requests`

- `requests` is a Python module that allows you to use Python to interact with the internet!  
- There are other packages that work similarly (e.g. `urllib`), but `requests` is arguably the easiest to use.

In [ ]:
import requests

### Example: `GET` requests via `requests`

To access the source code of the Bing home page, all we need to run is the following:

```py
requests.get('https://bing.com').text
```

In [ ]:
res = requests.get('https://bing.com')

`res` is now a `Response` object.

In [ ]:
res

In [ ]:
res.text

The `text` attribute of `res` is a string that containing the entire response.

In [ ]:
type(res.text)

In [ ]:
len(res.text)

In [ ]:
print(res.text[:1000])

### Example: `POST` requests via `requests`

The following call to `requests.post` makes a post request to https://httpbin.org/post, with a `'name'` parameter of `'King Triton'`.

In [ ]:
post_res = requests.post('https://httpbin.org/post',
                         data={'name': 'King Triton'})

post_res

In [ ]:
post_res.text

In [ ]:
# More on this shortly!
post_res.json()

What happens when we try and make a `POST` request somewhere where we're unable to?

In [ ]:
yt_res = requests.post('https://www.sjtu.edu.cn',
                       data={'name': 'King Triton'})

yt_res

`yt_res.text` is a string containing HTML – we can render this in-line using `IPython.display.HTML`.

In [ ]:
from IPython.display import HTML

In [ ]:
HTML(yt_res.text)

### HTTP status codes

- When we **request** data from a website, the server includes an **HTTP status code** in the response.  

* The most common status code is `200`, which means there were no issues.  

* Other times, you will see a different status code, describing some sort of event or error.
    - Common examples: `400` – bad request, `404` – page not found, `500` – internal server error.
    - [The first digit of a status describes its general "category".](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status)

- See [https://httpstat.us](https://httpstat.us/) for a list of all HTTP status codes.
    - It also has example sites for each status code; for example, https://httpstat.us/404 returns a `404`.

In [ ]:
yt_res.status_code

In [ ]:
r = requests.get('https://httpstat.us/200')
print(r.status_code)
print(r.text)

### Successful requests ✅

- You can check if a request was successful using the `ok` attribute, which returns a bool.
    - If a status is in the 200s, then it is successful.

In [ ]:
yt_res.status_code, yt_res.ok

In [ ]:
post_res.status_code, post_res.ok

- Unsuccessful requests can be re-tried, depending on the issue.
    - Wait a little, then try the request again.
    - You can even re-try requests programmatically (e.g. using a loop). If rate of requests is too high, slow down requests between each retry (e.g. using `time.sleep`).

## Data formats

### The data formats of the internet

Responses typically come in one of two formats: HTML or JSON.

- The response body of a `GET` request is usually either JSON (when using an API) or HTML (when accessing a webpage).

- The response body of a `POST` request is usually JSON.

- XML is also a common format, but not as popular as it once was.

### JSON

- JSON stands for **JavaScript Object Notation**. It is a lightweight format for storing and transferring data.

- It is:
    - very easy for computers to read and write.
    - moderately easy for programmers to read and write by hand.
    - meant to be generated and parsed.

- Most modern languages have an interface for working with JSON objects.
    - JSON objects _resemble_ Python dictionaries (but are not the same!).

### JSON data types

| Type | Description |
| --- | --- |
| String | Anything inside double quotes. |
| Number | Any number (no difference between ints and floats). |
| Boolean | `true` and `false`. |
| Null | JSON's empty value, denoted by `null`. |
| Array | Like Python lists. |
| Object | A collection of key-value pairs, like dictionaries. Keys must be strings, values can be anything (even other objects). |

See [json-schema.org](https://json-schema.org/understanding-json-schema/reference/type.html) for more details.

### Example JSON object

See `data/family.json`.

<center><img src='imgs/hierarchy.png' width=50%></center>

In [ ]:
import json

f = open(os.path.join('data', 'family.json'), 'r')
family_tree = json.load(f)

In [ ]:
family_tree

In [ ]:
family_tree['children'][0]['children'][0]['age']

In [ ]:
f_other = open(os.path.join('data', 'family.json'))
s = f_other.read()
s

In [ ]:
json.loads(s)

### Using the `json` module

Let's process the same file using the `json` module. Recall:
- `json.load(f)` loads a JSON file from a file object.
- `json.loads(f)` loads a JSON file from a **s**tring.

In [ ]:
f_other = open(os.path.join('data', 'evil_family.json'))
s = f_other.read()
s

In [ ]:
json.loads(s)

- Since `util.err()` is not a string in JSON (there are no quotes around it), `json.loads` is not able to parse it as a JSON object.

- This "safety check" is intentional.

### Handling _unfamiliar_ data

- Never trust data from an unfamiliar site.

- **Never** use `eval` on "raw" data that you didn't create!

- The JSON data format needs to be **parsed**, not evaluated as a dictionary.
    - It was designed with safety in mind!

## APIs and scraping

### Programmatic requests

* We learned how to use the Python `requests` package to exchange data via HTTP.
    - `GET` requests are used to request data **from** a server.
    - `POST` requests are used to **send** data to a server.

* There are two ways of collecting data through a request:
    * By using a published API (application programming interface).
    * By scraping a webpage to collect its HTML source code.

### APIs

An API is a service that makes data directly available to the user in a convenient fashion.

Advantages:

- The data are usually clean, up-to-date, and ready to use.

- The presence of a API signals that the data provider is okay with you using their data.

- The data provider can plan and regulate data usage.
    - Some APIs require you to create an API "key", which is like an account for using the API.
    - APIs can also give you access to data that isn't publicly available on a webpage.

Disadvantages:
- APIs don't always exist for the data you want!

### API terminology

- A URL, or uniform resource locator, describes the location of a website or resource.

- An **API endpoint** is a URL of the data source that the user wants to make requests to.

- For example, on the [Reddit API](https://www.reddit.com/dev/api/):
    * the `/comments` endpoint retrieves information about comments.
    * the `/hot` endpoint retrieves data about posts labeled "hot" right now. 
    - To access these endpoints, you add the endpoint name to the base URL of the API.

### API requests

- API requests are just `GET`/`POST` requests to a specially maintained URL.
- Let's test out the [Pokémon API](https://pokeapi.co).

First, let's make a `GET` request for `'squirtle'`.

In [ ]:
r = requests.get('https://pokeapi.co/api/v2/pokemon/squirtle')
r

Remember, the 200 status code is good! Let's take a look at the **content**:

In [ ]:
r.content[:1000]

Looks like JSON. We can extract the JSON from this request with the `json` method (or by passing `r.text` to `json.loads`).

In [ ]:
r.json()

Let's try a `GET` request for `'billy'`.

In [ ]:
r = requests.get('https://pokeapi.co/api/v2/pokemon/bil')
r

Uh oh...

### Scraping

Scraping is the act of programmatically "browsing" the web, downloading the source code (HTML) of pages that you're interested in extracting data from.

Advantages:

* You can always do it!
    - e.g. Google scrapes webpages in order to make them searchable.

Disadvantages:

- It is often difficult to parse and clean scraped data.
    - Source code often includes a lot of content unrelated to the data you're trying to find (e.g. formatting, advertisements, other text).

- Websites can change often, so scraping code can get outdated quickly.

- Websites may not want you to scrape their data!

- **In general, we prefer APIs.**

### Accessing HTML

**Goal**: Access information about JI research center.

Let's start by making a `GET` request to the JI research center page and see what the resulting HTML looks like. 

In [ ]:
r = requests.get('https://www.ji.sjtu.edu.cn/research/research-center/')
r

In [ ]:
research_text = r.text
len(research_text)

In [ ]:
print(research_text[:1000])

In [ ]:
'COO' in research_text

Wow, that is gross looking! 😰 

- It is **raw** HTML, which web browsers use to display websites.
- The information we are looking for – research information – is in there somewhere, but we have to search for it and extract it, which we wouldn't have to do if we had an API.
- We'll now look at how HTML documents are structured and how to extract information from them.

### Best practices for scraping

1. **Send requests slowly** and be upfront about what you are doing!
2. Respect the policy published in the page's `robots.txt` file.
    - Many sites have a `robots.txt` file in their root directory, which contains a policy that allows or disallows automatic access to their site. 
    - See [here](https://moz.com/learn/seo/robotstxt) for more details.
3. Don't spoof your User-agent (i.e. don't try to trick the server into thinking you are a person).
4. Read the Terms of Service for the site and follow it.

### Consequences of irresponsible scraping

If you make too many requests:
* The server may block your IP Address.
* You may take down the website.
   

## The anatomy of HTML documents

### What is HTML?

* HTML (HyperText Markup Language) is **the** basic building block of the internet. 


* It defines the content and layout of a webpage, and as such, it is what you get back when you scrape a webpage.

* See [this tutorial](http://fab.academany.org/2018/labs/fablaboshanghai/students/bob-wu/Fabclass/week2_project_management/HTML.html) for more details.

For instance, here's the content of a very basic webpage.

In [ ]:
!cat data/Lec8_ex.html

Using `IPython.display.HTML`, we can render it directly in our notebook.

In [ ]:
from IPython.display import HTML
HTML(os.path.join('data', 'Lec8_ex.html'))

### The anatomy of HTML documents

* **HTML document**: The totality of markup that makes up a webpage.

* **Document Object Model (DOM)**: The internal representation of a HTML document as a hierarchical **tree** structure.

* **HTML element**: An object in the DOM, such as a paragraph, header, or title.
* **HTML tags**: Markers that denote the **start** and **end** of an element, such as `<p>` and `</p>`.

<center><img src='imgs/dom.jpg'></center>

<center><a href='https://simplesnippets.tech/what-is-document-object-modeldom-how-js-interacts-with-dom/'>(source)</a></center>

### Useful tags to know


|Element|Description|
|:---|:---|
|`<html>`|the document|
|`<head>`|the header|
|`<body>`|the body|
|`<div>` |a logical division of the document|
|`<span>`|an *inline* logical division|
|`<p>`|a paragraph|
| `<a>`| an anchor (hyperlink)|
|`<h1>, <h2>, ...`| header(s) |
|`<img>`| an image |

There are many, many more. See [this article](https://en.wikipedia.org/wiki/HTML_element) for examples.

### Example: images and hyperlinks

Tags can have **attributes**, which further specify how to display information on a webpage.

For instance, `<img>` tags have `src` and `alt` attributes (among others):

```html
<img src="king-selfie.png" alt="A photograph of King Triton." width=500>
```

Hyperlinks have `href` attributes: 

```html
Click <a href="https://bing.com">this link</a> to search your keywords.
```

What do you think this webpage looks like?

In [ ]:
!cat data/Lec8.html

### The `<div>` tag

```html
<div style="background-color:lightblue">
  <h3>This is a heading</h3>
  <p>This is a paragraph.</p>
</div>
```

* The `<div>` tag defines a division or a "section" of an HTML document.
    * Think of a `<div>` as a "cell" in a Jupyter Notebook.

* The `<div>` element is often used as a container for other HTML elements to style them with CSS or to perform operations involving them using JavaScript.

* `<div>` elements often have attributes, **which are important when scraping**!

### Document trees

Under the document object model (DOM), HTML documents are trees. In DOM trees, child nodes are **ordered**.

<center>

<img src="imgs/webpage_anatomy.png" width="50%">

</center>    

What does the DOM tree look like for this document?

<center><img src="imgs/dom_tree.png" width="50%"></center>

### Example: Quote scraping

Consider the following webpage.

<center><img src="imgs/quotes2scrape.png" width=60%></center>

- What do you think the DOM tree looks like?
- If you had to store the data on this page in a DataFrame, what would the rows and columns represent?

<center><img src="imgs/quote_dom.png" width="50%"></center>

## Parsing HTML using Beautiful Soup

### Beautiful Soup 🍜

* [Beautiful Soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is a Python HTML parser.
    - To "parse" means to "extract meaning from a sequence of symbols".
* **Warning:** Beautiful Soup 4 and Beautiful Soup 3 work differently, so make sure you are using and looking at documentation for Beautiful Soup 4.

### Example HTML document

To start, we'll work with the source code for an HTML page with the DOM tree shown below:

<center><img src="imgs/dom_tree_1.png" width="50%"></center>

The string `html_string` contains an HTML "document".

In [ ]:
html_string = '''
<html>
    <body>
      <div id="content">
        <h1>Heading here</h1>
        <p>My First paragraph</p>
        <p>My <em>second</em> paragraph</p>
        <hr>
      </div>
      <div id="nav">
        <ul>
          <li>item 1</li>
          <li>item 2</li>
          <li>item 3</li>
        </ul>
      </div>
    </body>
</html>
'''.strip()

In [ ]:
HTML(html_string)

### `BeautifulSoup` objects

`bs4.BeautifulSoup` takes in a string or file-like object representing HTML (`markup`) and returns a **parsed** document.

In [ ]:
import bs4

In [ ]:
bs4.BeautifulSoup?

Normally, we pass the result of a `GET` request to `bs4.BeautifulSoup`, but here we will pass our hand-crafted `html_string`.

In [ ]:
soup = bs4.BeautifulSoup(html_string)
soup

In [ ]:
type(soup)

`BeautifulSoup` objects have several useful attributes, e.g. `text`:

In [ ]:
print(soup.text)

### Traversing through `descendants`

The `descendants` attribute traverses a `BeautifulSoup` tree using **depth-first traversal**.

Why depth-first? Elements closer to one another on a page are more likely to be related than elements further away.

<center><img src="imgs/dom_tree_1.png" width="60%"></center>

In [ ]:
soup.descendants

In [ ]:
list(soup.descendants)

In [ ]:
for child in soup.descendants:
    # print(child) # What would happen if we ran this instead?
    if isinstance(child, str):
        continue
    print(child.name)
    
    
# Traverse, BFS or DFS？


### Finding elements in a tree

Practically speaking, you will not use the `descendants` attribute (or the related `children` attribute) directly very often. Instead, you will use the following methods:

- `soup.find(tag)`, which finds the **first** instance of a tag (the first one on the page, i.e. the first one that DFS sees).
    - More general: `soup.find(name=None, attrs={}, recursive=True, text=None, **kwargs)`.
- `soup.find_all(tag)` will find **all** instances of a tag.


### Using `find`

Let's try and extract the first `<div>` subtree.

<center><img src="imgs/dom_tree_1.png" width="60%"></center>  

In [ ]:
soup

In [ ]:
div = soup.find('div')
div

<center><img src="imgs/dom_subtree_1.png" width="30%"></center>  

Let's try and find the `<div>` element that has an `id` attribute equal to `'nav'`.

In [ ]:
soup.find('div', attrs={'id': 'nav'})

`find` will return the first occurrence of a tag, regardless of its depth in the tree.

In [ ]:
soup.find('ul')

In [ ]:
soup.find('li')

### Node attributes
* The `text` attribute of a tag element gets the text between the opening and closing tags.
* The `attrs` attribute lists all attributes of a tag.
* The `get(key)` method gets the value of a tag attribute.

In [ ]:
soup.find('p')

In [ ]:
soup.find('p').text

In [ ]:
soup.find('div')

In [ ]:
soup.find('div').attrs

In [ ]:
soup.find('div').get('id')

The `get` method must be called directly on the node that contains the attribute you're looking for.

In [ ]:
soup

In [ ]:
# While there are multiple 'id' attributes, none of them are in the <html> tag at the top.
soup.get('id')

In [ ]:
soup.find('div').get('id')

### Using `find_all`

`find_all` returns a list of all matches.

In [ ]:
len(soup.find_all('div'))

In [ ]:
soup.find_all('li')[0]

In [ ]:
[x.text for x in soup.find_all('li')]

### Summary

- HTTP is the protocol the internet uses for transferring information.
- Clients can make `GET` HTTP requests to ask for information and `POST` HTTP requests to send information.
- Servers send responses with the desired information.
- We can use `curl` in the command-line or the `requests` Python module to make HTTP requests.
- The two main file formats used for storing information on the internet are HTML and JSON.
    - JSON objects resemble Python dictionaries, but they are not quite the same. 
    - Use the `.json()` method of a response object or the `json` package to parse them, **not** `eval`.
- APIs allow us to request information from web servers in a convenient fashion.
- When APIs don't exist, we instead scrape webpages to access their source HTML and then parse the HTML to extract the information we care about.
- Under the document object model (DOM), HTML documents are trees.
    - Elements are defined by tags.
- Beautiful Soup is an HTML parser that allows us to (somewhat) easily extract information from HTML documents.
    - `soup.find` and `soup.find_all` are the functions you will use most often.

### Next time

- Using HTTP to make API requests and scrape the web. 
- Parsing HTML files.